In [2]:
import os

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.linear_model import LinearRegression

import ProjectLibrary as pl

Setting up hyperparameters version - model version, testing - debug or output.

In [12]:
version = 2
testing = 0

Loading Data

In [5]:
path = "DataSets/"
DataSet_A = pd.read_csv(path + 'Study_A.csv').iloc[:,0:39]
DataSet_B = pd.read_csv(path + 'Study_B.csv').iloc[:,0:39]
DataSet_C = pd.read_csv(path + 'Study_C.csv').iloc[:,0:39]
DataSet_D = pd.read_csv(path + 'Study_D.csv').iloc[:,0:39]
DataSet_E = pd.read_csv(path + 'Study_E.csv').iloc[:,0:39]

Sample_Sub = pd.read_csv("DataSets/sample_submission_PANSS.csv")

DataSet_All = pd.concat([DataSet_A, DataSet_B, DataSet_C, DataSet_D, DataSet_E])

Creating test & train data - our model predicts the day of last visit and the PANSS_Total

In [6]:
DataSet_E_Test = pd.DataFrame(columns=DataSet_E.columns)
DataSet_E_Train = pd.DataFrame(columns=DataSet_E.columns)
DataSet_E_AllOutput = DataSet_E.copy()
PatientID = -1
row_n = -1

while row_n + 1 < DataSet_E.shape[0]:
    if PatientID != DataSet_E.iloc[row_n + 1]["PatientID"]:
        if PatientID != -1:
            DataSet_E_Test = DataSet_E_Test.append(DataSet_E.iloc[row_n])
        PatientID = DataSet_E.iloc[row_n]["PatientID"]
    else:
        DataSet_E_Train = DataSet_E_Train.append(DataSet_E.iloc[row_n])
        
    print("Done %8.2f%%" % ((row_n+1)/DataSet_E.shape[0] * 100)) if not row_n % 100 else None

    row_n += 1
    
DataSet_E_Test = DataSet_E_Test[DataSet_E_Test["VisitDay"] > 0]

DataSet_E_Test = DataSet_E_Test.reset_index()
DataSet_E_Train = DataSet_E_Train.reset_index()
DataSet_E_AllOutput = DataSet_E_AllOutput.reset_index()

Sel_E_Output = DataSet_E_AllOutput[DataSet_E_AllOutput["PatientID"].isin(Sample_Sub["PatientID"])]
    
print("Finished!")

Done     0.05%
Done     5.15%
Done    10.24%
Done    15.34%
Done    20.44%
Done    25.54%
Done    30.63%
Done    35.73%
Done    40.83%
Done    45.92%
Done    51.02%
Done    56.12%
Done    61.21%
Done    66.31%
Done    71.41%
Done    76.50%
Done    81.60%
Done    86.70%
Done    91.79%
Done    96.89%
Finished!


Predicting for the test dataset. Works if testing = 1

In [13]:
Predict_E = pl.SimpleUniqueLinear(DataSet_E_Train.copy()) if testing else None

--------------------
Progress:     1.34%
--------------------
Progress:     2.67%
--------------------
Progress:     4.01%
--------------------
Progress:     5.35%
--------------------
Progress:     6.68%
--------------------
Progress:     8.02%
--------------------
Progress:     9.36%
--------------------
Progress:    10.70%
--------------------
Progress:    12.03%
--------------------
Progress:    13.37%
--------------------
Progress:    14.71%
--------------------
Progress:    16.04%
--------------------
Progress:    17.38%
--------------------
Progress:    18.72%
--------------------
Progress:    20.05%
--------------------
Progress:    21.39%
--------------------
Progress:    22.73%
--------------------
Progress:    24.06%
--------------------
Progress:    25.40%
--------------------
Progress:    26.74%
--------------------
Progress:    28.07%
--------------------
Progress:    29.41%
--------------------
Progress:    30.75%
--------------------
Progress:    32.09%
----------------

Predicting for the final output

In [ ]:
Sel_Predict_E_Output = pl.SimpleUniqueLinear(Sel_E_Output.copy())

Calculates the RSS error for the test set

In [14]:
 if testing:
    RSS_1 = 0
    counter = 1
    for PatientID in DataSet_E_Test["PatientID"].unique():
        if (Predict_E["PatientID"] == PatientID).sum() != 0:
    #         print(RSS_1)
    #         print((float(DataSet_E_Test.loc[DataSet_E_Test["PatientID"] == PatientID, "PANSS_Total"]) - float(Predict_E.loc[Predict_E["PatientID"] == PatientID, "Predict"])) ** 2)
            RSS_1 += ((float(DataSet_E_Test.loc[DataSet_E_Test["PatientID"] == PatientID, "PANSS_Total"].mean()) - float(Predict_E.loc[Predict_E["PatientID"] == PatientID, "PANSS_Total"].mean())) ** 2)
    #         print("PatientID:{} RSS_1:{}".format(PatientID, RSS_1 / counter))
            counter += 1

    RSS_1 = RSS_1 / len(Predict_E["PatientID"].unique())
    print(RSS_1)

70.82438135696114


In [ ]:
pl.CompRSS_Previous(Sel_Predict_E_Output, version)

In [ ]:
version += 1
Sel_Predict_E_Output.to_csv("prediction_{}.csv".format(version), index=False)

Version 1: Simple prediction using k=1 NN if less than 2 observations, otherwise using LinearRegression
Version 2: Prediction using k=5 NN, avaraging with LinearRegression if more than 1 observations - int ouput
Version 3: 